## 使用 GPT-4 构造数据

ChatGPT 分享链接: https://chat.openai.com/c/e8e6ecda-4291-44fc-9920-5776e41acf20


使用 ChatGPT 的 Prompt 示例：

```
假设你是一位博学的妈妈，你有一个五岁的宝宝，你经常要回答一些宝宝的问题，请模拟 10 条对话，宝宝问一句，妈妈答一句。分别以 [宝宝] 和 [妈妈] 开头，每次对话的两句不加空行，对话结束后加两行空行
```

GPT-3.5 回复结果：

```
[宝宝]：妈妈，为什么天空是蓝色的？
[妈妈]：天空是蓝色的因为光在大气中散射时，蓝色光散射得最多，所以我们看到的是蓝天。

[宝宝]：哦，那晚上天空是黑色的是因为没有光吗？
[妈妈]：对的，晚上太阳不再照亮天空，所以我们看到黑色的夜空，但还有很多星星点缀着。

[宝宝]：星星是什么？
[妈妈]：星星是遥远的太阳，它们发出的光照亮了夜晚的天空。它们就像天上的小灯泡一样。

[宝宝]：为什么月亮不是太阳呢？
[妈妈]：因为月亮是地球的卫星，它围绕地球旋转，而太阳是一个恒星，位于太阳系的中心。

[宝宝]：妈妈，月亮为什么会有不同的形状？
[妈妈]：月亮有不同的形状是因为我们看到的部分是太阳照亮的部分，月球围绕地球运动，所以我们每天都能看到不同的形状。

[宝宝]：为什么我们要吃蔬菜？
[妈妈]：吃蔬菜是因为它们对我们的身体很好。蔬菜里有各种维生素和矿物质，能让我们保持健康，而且它们很好吃！

[宝宝]：为什么树叶会变颜色？
[妈妈]：树叶变颜色是因为在秋天，树木停止制造绿色叶子的色素，而橙色和红色的色素开始显现，所以我们看到了五彩斑斓的叶子。

[宝宝]：妈妈，为什么鱼生活在水里？
[妈妈]：鱼生活在水里因为它们需要水来呼吸和生存。水中有氧气，对鱼来说就像我们呼吸空气一样重要。

[宝宝]：妈妈，我们为什么要刷牙？
[妈妈]：我们要刷牙是因为每天吃东西后，嘴里会留下细菌，刷牙可以帮助我们保持口腔卫生，防止蛀牙和口臭。

[宝宝]：为什么水是透明的？
[妈妈]：水是透明的因为它不会阻挡光线，光线可以穿透水分子，所以我们可以看到水的下面。
```


## 注意，我这里自己在原数据中又每个对话前面加了一个空行，这样就可以使用 CharacterTextSplitter 来进行文本分割了
## chatgpt 的回答对加空行的指令执行得不好
## 使用 Document Transformers 模块来处理原始数据


将 ChatGPT 生成的结果保存到 [chidlren_whys_data.txt](children_whys_data.txt) 文件中

In [2]:
with open("children_whys_data.txt") as f:
    children_whys_data = f.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [3]:
from langchain.text_splitter import CharacterTextSplitter

In [4]:
from platform import python_version
print(python_version())

3.10.3


In [5]:
text_splitter = CharacterTextSplitter(
    separator = r'\n\n',
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [6]:
docs = text_splitter.create_documents([children_whys_data])

In [7]:
docs[1]

Document(page_content='[宝宝]：哦，那晚上天空是黑色的是因为没有光吗？\n[妈妈]：对的，晚上太阳不再照亮天空，所以我们看到黑色的夜空，但还有很多星星点缀着。', metadata={})

In [8]:
len(docs)

87

### 使用 Faiss 作为向量数据库，持久化亲子问答对（QA-Pair）

In [14]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings())

In [15]:
query = "为什么只有白天能看到太阳"

In [16]:
answer_list = db.similarity_search(query)

In [17]:
for ans in answer_list:
    print(ans.page_content + "\n")

[宝宝]：妈妈，为什么月亮只在晚上出来？
[妈妈]：月亮只在晚上出来是因为它的亮度不足以在白天看到，所以我们通常在夜里太阳落下才能看到它。

[宝宝]：妈妈，为什么月亮会有阴晴圆缺？
[妈妈]：月亮有阴晴圆缺是因为我们看到的是月球不同位置的光照面，有时候太阳照在月球正面，有时候只照在一部分上面。

[宝宝]：妈妈，为什么月亮会有阴晴圆缺？
[妈妈]：月亮有阴晴圆缺是因为我们看到的是月球不同位置的光照面，有时候太阳照在月球正面，有时候只照在一部分上面。

[宝宝]：妈妈，为什么夜晚会有月亮？
[妈妈]：夜晚有月亮是因为月球绕地球旋转，它的光亮一部分被反射到地球上，所以我们在夜晚能看到它。



In [18]:
db.save_local("children_whys")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [19]:
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [20]:
topK_retriever

VectorStoreRetriever(tags=['FAISS'], metadata=None, vectorstore=<langchain.vectorstores.faiss.FAISS object at 0x117a27c10>, search_type='similarity', search_kwargs={'k': 3})

In [21]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[宝宝]：妈妈，为什么月亮只在晚上出来？
[妈妈]：月亮只在晚上出来是因为它的亮度不足以在白天看到，所以我们通常在夜里太阳落下才能看到它。

[宝宝]：妈妈，为什么月亮会有阴晴圆缺？
[妈妈]：月亮有阴晴圆缺是因为我们看到的是月球不同位置的光照面，有时候太阳照在月球正面，有时候只照在一部分上面。

[宝宝]：妈妈，为什么月亮会有阴晴圆缺？
[妈妈]：月亮有阴晴圆缺是因为我们看到的是月球不同位置的光照面，有时候太阳照在月球正面，有时候只照在一部分上面。



In [22]:
docs = topK_retriever.get_relevant_documents("彩虹是怎么产生的")

In [23]:
for doc in docs:
    print(doc.page_content + "\n")

[宝宝]：妈妈，为什么雨后会有彩虹？
[妈妈]：雨后会有彩虹是因为雨滴在空中散射太阳光，形成了彩虹。雨后太阳出现，彩虹才能出现在天空中。

[宝宝]：妈妈，什么是彩虹？
[妈妈]：彩虹是太阳光经过雨滴折射和反射形成的，所以我们看到了不同颜色的光组成的弧形。

[宝宝]：妈妈，什么是彩虹？
[妈妈]：彩虹是太阳光经过雨滴折射和反射形成的，所以我们看到了不同颜色的光组成的弧形。



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [24]:
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)

In [25]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[宝宝]：妈妈，为什么月亮只在晚上出来？
[妈妈]：月亮只在晚上出来是因为它的亮度不足以在白天看到，所以我们通常在夜里太阳落下才能看到它。



### 提取向量数据库中的`销售回答`

In [26]:
docs = retriever.get_relevant_documents(query)

In [27]:
docs[0].page_content

'[宝宝]：妈妈，为什么月亮只在晚上出来？\n[妈妈]：月亮只在晚上出来是因为它的亮度不足以在白天看到，所以我们通常在夜里太阳落下才能看到它。'

In [28]:
docs[0].page_content.split("[妈妈]：")

['[宝宝]：妈妈，为什么月亮只在晚上出来？\n', '月亮只在晚上出来是因为它的亮度不足以在白天看到，所以我们通常在夜里太阳落下才能看到它。']

In [29]:
ans = docs[0].page_content.split("[妈妈]：")[-1]

In [30]:
ans

'月亮只在晚上出来是因为它的亮度不足以在白天看到，所以我们通常在夜里太阳落下才能看到它。'

#### 尝试各种问题

In [31]:
from typing import List

def mom_answers(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[妈妈]：")[-1] for doc in docs]

    return ans_list

In [34]:
query = "月亮什么时候圆"

print(mom_answers(query))

['月亮有阴晴圆缺是因为我们看到的是月球不同位置的光照面，有时候太阳照在月球正面，有时候只照在一部分上面。', '月亮有阴晴圆缺是因为我们看到的是月球不同位置的光照面，有时候太阳照在月球正面，有时候只照在一部分上面。', '夜晚有月亮是因为月球绕地球旋转，它的光亮一部分被反射到地球上，所以我们在夜晚能看到它。']


In [35]:
print(mom_answers(query, 0.75))

['月亮有阴晴圆缺是因为我们看到的是月球不同位置的光照面，有时候太阳照在月球正面，有时候只照在一部分上面。', '月亮有阴晴圆缺是因为我们看到的是月球不同位置的光照面，有时候太阳照在月球正面，有时候只照在一部分上面。', '夜晚有月亮是因为月球绕地球旋转，它的光亮一部分被反射到地球上，所以我们在夜晚能看到它。', '因为月亮是地球的卫星，它围绕地球旋转，而太阳是一个恒星，位于太阳系的中心。']


In [39]:
query = "小朋友生病了一定要去医院吗"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

score:0.8 ans: []

score:0.75 ans: []

score:0.5 ans: ['我们穿衣服是为了保暖，保护身体免受寒冷和天气的影响。不同季节需要穿不同的衣物。', '有人打呼噜是因为当他们睡觉时，喉咙和气道的肌肉松弛，空气通过时会产生声音。', '吃蔬菜是因为它们对我们的身体很好。蔬菜里有各种维生素和矿物质，能让我们保持健康，而且它们很好吃！', '对的，晚上太阳不再照亮天空，所以我们看到黑色的夜空，但还有很多星星点缀着。']



#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [40]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [41]:
qa_chain({"query": query})

{'query': '小朋友生病了一定要去医院吗',
 'result': '如果小朋友生病了，最好去医院就诊。医院有专业的医生和设备，可以对病情进行准确的诊断和治疗。但是，如果病情不严重，可以先尝试在家休息和进行简单的自我护理。如果病情加重或持续不好，建议尽快就医。'}

In [42]:
qa_chain({"query": "彩虹什么时候出现"})

{'query': '彩虹什么时候出现',
 'result': '彩虹通常在雨后太阳出现的时候出现在天空中。当太阳的光线经过雨滴时，光线会被雨滴散射和折射，形成彩虹的光谱。所以，只有在雨后太阳出现的时候，我们才能看到彩虹。'}

In [43]:
print(mom_answers("彩虹什么时候出现"))

['雨后会有彩虹是因为雨滴在空中散射太阳光，形成了彩虹。雨后太阳出现，彩虹才能出现在天空中。']


## 加载 FAISS 向量数据库已有结果

In [44]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.load_local("children_whys", OpenAIEmbeddings())

In [45]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [46]:
qa_chain({"query": "妈妈，嫦娥和后羿的传说是什么"})

{'query': '妈妈，嫦娥和后羿的传说是什么',
 'result': '嫦娥和后羿是中国传说中的两个角色。嫦娥是一个美丽的仙女，据说她嫁给了后羿，一个英勇的射手。传说中，后羿曾经射下了九个太阳，拯救了人们免受炎热的侵扰。作为奖励，他得到了一瓶长生不老的仙药。然而，后羿的妻子嫦娥偷偷喝下了这瓶仙药，变成了一个仙女，并飞到了月亮上。这就是嫦娥和后羿的传说。'}

In [49]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = False

In [50]:
qa_chain({"query": "妈妈，嫦娥和后羿的传说是什么"})

{'query': '妈妈，嫦娥和后羿的传说是什么',
 'result': '嫦娥和后羿是中国传说中的两个角色。嫦娥是一个美丽的仙女，据说她嫁给了后羿，一个英勇的射手。传说中，后羿曾经射下了九个太阳，拯救了人们免受炎热的侵扰。作为奖励，他得到了一瓶长生不老的仙药。然而，后羿的妻子嫦娥偷偷喝下了这瓶仙药，变成了一个仙女，并飞到了月亮上。这就是嫦娥和后羿的传说。'}

In [51]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [53]:
result = qa_chain({"query": "妈妈，你什么都知道吗"})

In [55]:
result

{'query': '妈妈，你什么都知道吗',
 'result': '妈妈可能不知道所有的事情，但她会尽力回答你的问题。有些问题可能超出她的知识范围，她可能会告诉你她不知道。',
 'source_documents': []}